In [20]:
import os
import requests
from lxml import etree
import pandas as pd
import re
import json
from selenium import webdriver
import time
import datetime
import config
import public_fun

In [21]:
def handle(url, s_date):
    res = requests.get(url=url, headers=config.HEADERS, verify=False)
    if res.status_code != 404:
        res.encoding = res.apparent_encoding
        html = etree.HTML(res.text)
    else:
        print('url1 404错误:', res.status_code, url)
        return [], ''
    urls = html.xpath('//*[@class="box-result clearfix"]/h2/a/@href')
    labels = html.xpath('//*[@class="box-result clearfix"]/h2/span/text()') # 获取时间
    dates = [l.split(' ')[1] for l in labels]

    index_next_page = 'https://search.sina.com.cn/{}'
    next_href = index_next_page.format(html.xpath('//*[@title="下一页"]/@href')[0])

    if len(urls) != len(dates):
        print('链接长度:', len(urls), '时间长度:', len(dates))
        return False
    else:
        # index_url2 = 'http://www.51pdf.cn{}'
        result = [urls[n] for n in range(len(urls)) if public_fun.calc_date(s_date=s_date, e_date=dates[n])]  # 通过时间过滤url2
        return result, next_href

def get_data(url, search_word, max_text):
    json_result = {'source': 'xl', 'doc_id': '', 'date': '', 'download_url': '',
                   'org_name': '', 'page_num': '1', 'doc_type': 'NEW', 'title': ''}
    path = os.path.join(config.SAVE_PATH, search_word, 'news', 'xinlang')  # 路径
    res = requests.get(url=url, headers=config.HEADERS)
    res.encoding = res.apparent_encoding
    html = etree.HTML(res.text)
    content_text = public_fun.filter_space_json(html.xpath('//*[@id="artibody"]//text()')) # 正文内容

    if len(content_text) > max_text:
        try:
            # ==============================html=========================
            # doc_id = re.findall('\d+', string=url)[0]  # 文章ID
            doc_id = url.split('/')[-1].split('.')[0] # 文章ID
            title = html.xpath('//*[@class="main-title"]')[0]    # 标题
            description = html.xpath('//*[@id="artibody"]')[0]   #文章内容 html
            html_list = [title, description]
            html_result = public_fun.filter_space_html(html_list)
            # ==============================json=========================
            json_result['doc_id'] = doc_id
            try:
                json_result['date'] = public_fun.filter_space_json(html.xpath('//*[@id="pub_date"]/text()')[0].replace('-', ''))[:10]\
                    .replace('年', '').replace('月', '')
            except:
                json_result['date'] = public_fun.filter_space_json(html.xpath('//*[@class="date"]/text()')[0].replace('-', ''))[:10]\
                    .replace('年', '').replace('月', '')
            json_result['download_url'] = url
            json_result['org_name'] = public_fun.filter_space_json(html.xpath('//*[@id="author_ename"]/text()'))
            json_result['title'] = public_fun.filter_space_json(html.xpath('//*[@id="artibodyTitle"]/text()'))
            public_fun.write_down_json(path=path, filename=doc_id + '.json', text=json_result)
            public_fun.write_down_html(path=path, filename=doc_id + '.html', text=html_result)
        except Exception as e:
            print(e.__traceback__.tb_lineno, url, e)
    else:
        print('文章字数不足', max_text,url)
        
def main(search_word, max_art, max_text, s_date):
    url1 = 'https://search.sina.com.cn/?country=usstock&q={}&c=news'.format(search_word)  # 文章url列表
    url2_list = []  # 文章内容url
    while url1:
        one_page_url, next_page = handle(url=url1, s_date=s_date)
        url2_list += one_page_url
        if next_page and len(url2_list) < max_art:
            # index_url = 'https://s.iresearch.cn/'   # url2域名
            url1 = next_page
        else:
            url1 = ''
    # return url2_list
    url2_list = url2_list[:5] if config.DEBUG else url2_list   # 取十条测试数据
    for url2 in url2_list:
        get_data(url=url2, search_word=search_word, max_text=max_text)


In [22]:
p0 = '人工智能'
p1 = 10
p2 = 500
p3 = '2018-01-01'
r = main(search_word=p0, max_art=p1, max_text=p2, s_date=p3)


c:\python\miniconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
c:\python\miniconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


文章字数不足 500 https://k.sina.com.cn/article_5044281310_12ca99fde02001e4wt.html?from=cul


In [23]:
# 请求测试部分
# test_url = 'http://tech.sina.com.cn/csj/2020-09-28/doc-iivhuipp6997654.shtml'
# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36',
# }
# test_res = requests.get(url=test_url, headers=headers)
# test_res.encoding = test_res.apparent_encoding
# h = etree.HTML(test_res.text)
# title = h.xpath('//*[@class="box-result clearfix"]/h2/a/@href')
# for n in title:
#     print(n)
# test_string = '喂式信息流造就'
# print(test_string in test_res.text)
# with open('test_check', 'w', encoding='utf-8') as f:
#     f.write(test_res.text)

In [24]:
# 其他测试部分
# a = 'https://www.ofweek.com/ai/2020-09/ART-201713-8470-30458368_2.html'
# b = '/'.join(a.split('/')[:-2])
# print(b)
